In [2]:
import tensorflow as tf
import tensorflow.keras.layers as kl

2023-09-16 16:11:21.890613: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-16 16:11:21.892276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 16:11:21.922813: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 16:11:21.923656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 16:11:22.381930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [3]:
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, channels, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        self.W_gate = tf.keras.Sequential(
            [ kl.Conv2D(channels, 1, 1, padding="same"), 
             kl.BatchNormalization()]
        )
        self.W_x = tf.keras.Sequential(
            [ kl.Conv2D(channels, 1, 1, padding="same"),
                kl.BatchNormalization()]
        )
        self.phi = tf.keras.Sequential(
            [ kl.Conv2D(channels, 1, 1, padding="same"),
                kl.BatchNormalization()]
        )

        self.attention = kl.Attention()

    def call(self, x, g ):
        g = self.W_gate(g)
        x1 = self.W_x(x)
        phi = g + x1
        phi = kl.Activation("relu")(phi)
        phi = self.phi(phi)
        phi = kl.Activation("sigmoid")(phi)

        phi = self.attention([x, phi])
        return phi * x


In [5]:
layer = SelfAttention(64)

x = tf.random.normal((1, 32, 32, 64))
g = tf.random.normal((1, 32, 32, 12))

layer(x, g).shape

TensorShape([1, 32, 32, 64])